In [1]:
import numpy as np
import pandas as pd

from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr
    
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [3]:
ds = load_dataset("beanham/spatial_join_dataset")
test=ds['test']
gt=np.array(test['label'])

In [4]:
results=[]
metric_values = ['worst_single', 'best_single', 'worst_comb', 'best_comb', 'worst_all', 'best_all']
for model in ['4o_mini', 'qwen_plus']:
    for value in metric_values:
        few_shot=np.load(f'base/{model}_correction/{model}_{value}_few_shot_with_heur_value_all_correction.npy')
        few_shot=post_processing(few_shot)
        few_metrics=metric_calculation(few_shot, gt)        
        results.append([model, value, 'few_shot', few_metrics[0], few_metrics[1]])
results=pd.DataFrame(results, columns=['model', 'value', 'prompt', 'acc', 'f1'])        

In [5]:
results

,model,value,prompt,acc,f1
0,4o_mini,worst_single,few_shot,0.945,0.941365
1,4o_mini,best_single,few_shot,0.966,0.964271
2,4o_mini,worst_comb,few_shot,0.957,0.954532
3,4o_mini,best_comb,few_shot,0.952,0.949269
4,4o_mini,worst_all,few_shot,0.961,0.958802
5,4o_mini,best_all,few_shot,0.955,0.952417
6,qwen_plus,worst_single,few_shot,0.957,0.955766
7,qwen_plus,best_single,few_shot,0.974,0.972601
8,qwen_plus,worst_comb,few_shot,0.950,0.948753
9,qwen_plus,best_comb,few_shot,0.988,0.987468
